## AVL Trees

**Note**: Có thể kiểm tra nhanh các hàm dưới đây bằng cách em visualization theo link [này](https://www.cs.usfca.edu/~galles/visualization/AVLtree.html)

<ins>**Định nghĩa**</ins>: Cây AVL là một cây Binary Search Tree với các đặc điểm:**Tại mỗi đỉnh của cây, độ cao của cây con trái và cây con phải không chênh lệch quá 1.**
       

<ins>**Mục đích**</ins>: Không để một cây BST suy biến thành một **LinkedList**, để quá trình tìm kiếm trên BST luôn tốt trong mọi trường hợp.

### 1. Hệ số cân bằng
Hệ số cân bằng của cây T là hiệu số  giữa các chiều cao của cây con trái và cây con phải của nỏ.

Ký hiệu hệ số cân bằng của cây con gốc u là **balance(u)**. Hệ số cân bằng của cây T là **balance(T)**

$$balance(u) = height(u.left) - height(u.right)$$

### 2. Các trường hợp mất cân bằng

* Mất cân bằng trái trái (L-L):
![alt](LL.png)
* Mất cân bằng trái phải (L-R):
![alt](LR.png)
* Mất cân bằng phải phải (R-R):
![alt](RR.png)
* Mất cân bằng phải trái (R-L):
![alt](RL.png)

### 3. Xây dựng cây cân bằng

* Việc xây dựng cây cân bằng dựa trên BST, chỉ bổ sung thêm 1 giá trị cho biết sự cân bằng của các cây con như thế nào.

* Trong đó giá trị cân bằng (balance) có thể  là 0: cân bằng, lệch trái hoặc lệch phải.

In [64]:
class Node(object):
    def __init__(self, value):
        self.value = value
        self.right = None
        self.left = None
        self.parent = None
        self.height = 1
        
class AVLTree(object):
    # Khởi tạo với root
    def __init__(self, root=None):
        self.root = Node(root)
        
    def insert(self, value):
        node = Node(value)
        '''
        Sau khi insert 1 node mới thì cập nhật lại height cho các node
        parent của node đó, trong quá trình cập nhật thì kiểm tra độ
        cân bằng của từng nốt parent -> nếu abs(balance) = 2 thì cây
        mất cân bằng -> cân bằng lại cây
        '''
        self.insert_helper(self.root, node)
        self.update_height(node)
            
    def insert_helper(self, current, node):
        '''
        Hàm hỗ trợ này sẽ đi theo thứ tự của cây BST
        Bé thua parent thì sẽ đi về bên trái, ngược lại sẽ đi về  phải.
        Đi cho đến khi 1 vị trí current nào đó đang None thì điền node
        này vào chỗ đó, cùng lúc đó thì gắn parent cho nó luôn.
        '''
        if current:
            if node.value < current.value:
                if current.left == None:
                    current.left = node
                    node.parent = current
                else:
                    self.insert_helper(current.left, node)
            else:
                if current.right == None:
                    current.right = node
                    node.parent = current
                else:
                    self.insert_helper(current.right, node)
        else:
            current = node
            node.parent = current
    
    def balance(self, node):
        '''
        Tính độ cân bằng tại 1 node bất kỳ. Độ cân bằng của 1 node
        chính là chiều cao của node con bên trái trừ đi chiều cao của 
        node con bên phải.
        
        Trường hợp 1 node là None thì nó chỉ có 1 node con là root vì 
        vậy trả về giá trị 0
        '''
        if node == None:
            return 0
        if node.left == None:
            height_left = 0
        else:
            height_left = node.left.height
        
        if node.right == None:
            height_right = 0
        else:
            height_right = node.right.height
        
        return height_left - height_right
    
    def update_height(self, node):
        '''
        Vì mình chỉ cập nhật chiều cao cho các node parent khi
        chiều cao của node con bằng chiều cao của nó.

        Từ đó dẫn tới việc khi đệ quy thì nó quy về luôn cả root.
        Để tránh trường hợp tính balance bị lỗi vì None value, mình 
        sẽ thoát khỏi vòng đệ quy cho nó.
        '''
        if node == self.root:
            return 
        if node.height == node.parent.height:
            node.parent.height += 1
            balance = self.balance(node.parent.parent)
            '''
            Cây sẽ bị mất cân bằng khi đọ chênh lệch chiều cao lớn hơn
            hoặc bằng 2.
            '''
            if balance >= 2:
                if self.balance(node.parent) == 1:
                    print("LL")
                    self.rotate(node.parent, "LL")
                    return
                elif self.balance(node.parent) == -1:
                    print("LR")
                    self.rotate(node.parent, "LR")
                    return
            elif balance <= -2:
                if self.balance(node.parent) == -1:
                    print("RR")
                    self.rotate(node.parent, "RR")
                    return
                elif self.balance(node.parent) == 1:
                    print("RL")
                    self.rotate(node.parent, "RL")
                    return
            self.update_height(node.parent)

    def rotate(self, node, case):
        '''
        Cân bằng lại cây theo hình bên dưới.
        '''
        if case == "LL":
            temp = node.parent
            if temp.parent:
                temp.parent.left = node
            else:
                self.root = node
            node.right = temp
            temp.parent = node
            temp.height -= 1
            temp.left = None
            
        if case == "LR":
            node.parent.left = node.right
            node.right.parent = node.parent
            node.right.left = node
            node.right.height += 1
            node.parent = node.right
            node.right = None
            node.height -= 1
            self.rotate(node.parent, "LL")
        
        if case == "RR":
            temp = node.parent
            if temp.parent:
                temp.parent.right = node
            else:
                self.root = node
            node.left = temp
            temp.parent = node
            temp.height -= 1
            temp.right  = None
            
        if case == "RL":
            node.parent.right = node.left
            node.left.parent = node.parent
            node.left.right = node
            node.left.height += 1
            node.parent = node.left
            node.left = None
            node.height -= 1
            self.rotate(node.parent, "RR")
            
    def print_dfs(self):
        '''
        Trả về 1 string traversal là đường đi qua các node trong cây.
        '''
        return self.print_helper(self.root, "")[:-1]
    
    def print_helper(self, start, traversal):
        '''
        Đệ quy sử dụng thuật toán DFS(Depth First Search) 
        để print hết các node trong Tree.
        '''
        if start:
            traversal = self.print_helper(start.left, traversal)
            traversal += str(start.value) + "(%d)"%start.height + "-"
            traversal = self.print_helper(start.right, traversal)
        return traversal

### 4. Rotation

* LL Rotation
![alt](LL_rotate.png)

* LR Rotation
![alt](LR_rotate.png)

* RR Rotation
![alt](RR_rotate.png)

* RL Rotation
![alt](RL_rotate.png)

### Test Trường hợp cây bị mất cân bằng LL
* Mất cân bằng trái trái (L-L):
![alt](LL.png)

In [65]:
tree = AVLTree(12)

In [66]:
tree.insert(8)
tree.print_dfs()

'8(1)-12(2)'

In [67]:
tree.insert(18)
tree.print_dfs()

'8(1)-12(2)-18(1)'

In [68]:
tree.insert(5)
tree.print_dfs()

'5(1)-8(2)-12(3)-18(1)'

In [69]:
tree.insert(17)
tree.print_dfs()

'5(1)-8(2)-12(3)-17(1)-18(2)'

In [70]:
tree.insert(4)
tree.print_dfs()

LL


'4(1)-5(2)-8(1)-12(3)-17(1)-18(2)'

##### Kết quả đúng

### Test Trường hợp cây bị mất cân bằng LR
* Mất cân bằng trái phải (L-R):
![alt](LR.png)

In [71]:
tree = AVLTree(12)

In [72]:
tree.insert(8)
tree.print_dfs()

'8(1)-12(2)'

In [73]:
tree.insert(18)
tree.print_dfs()

'8(1)-12(2)-18(1)'

In [74]:
tree.insert(5)
tree.print_dfs()

'5(1)-8(2)-12(3)-18(1)'

In [75]:
tree.insert(17)
tree.print_dfs()

'5(1)-8(2)-12(3)-17(1)-18(2)'

In [76]:
tree.insert(7)
tree.print_dfs()

LR


'5(1)-7(2)-8(1)-12(3)-17(1)-18(2)'

##### Kết quả đúng

### Test Trường hợp cây bị mất cân bằng RR
* Mất cân bằng phải phải (R-R):
![alt](RR.png)

In [77]:
tree = AVLTree(12)

In [78]:
tree.insert(8)
tree.print_dfs()

'8(1)-12(2)'

In [79]:
tree.insert(18)
tree.print_dfs()

'8(1)-12(2)-18(1)'

In [80]:
tree.insert(5)
tree.print_dfs()

'5(1)-8(2)-12(3)-18(1)'

In [81]:
tree.insert(11)
tree.print_dfs()

'5(1)-8(2)-11(1)-12(3)-18(1)'

In [82]:
tree.insert(22)
tree.print_dfs()

'5(1)-8(2)-11(1)-12(3)-18(2)-22(1)'

In [83]:
tree.insert(4)
tree.print_dfs()

'4(1)-5(2)-8(3)-11(1)-12(4)-18(2)-22(1)'

In [84]:
tree.insert(7)
tree.print_dfs()

'4(1)-5(2)-7(1)-8(3)-11(1)-12(4)-18(2)-22(1)'

In [85]:
tree.insert(25)
tree.print_dfs()

RR


'4(1)-5(2)-7(1)-8(3)-11(1)-12(4)-18(1)-22(2)-25(1)'

##### Kết quả đúng

### Test Trường hợp cây bị mất cân bằng RL
* Mất cân bằng phải trái (R-L):
![alt](RL.png)

In [86]:
tree = AVLTree(12)

In [87]:
tree.insert(8)
tree.print_dfs()

'8(1)-12(2)'

In [88]:
tree.insert(18)
tree.print_dfs()

'8(1)-12(2)-18(1)'

In [89]:
tree.insert(5)
tree.print_dfs()

'5(1)-8(2)-12(3)-18(1)'

In [90]:
tree.insert(11)
tree.print_dfs()

'5(1)-8(2)-11(1)-12(3)-18(1)'

In [91]:
tree.insert(22)
tree.print_dfs()

'5(1)-8(2)-11(1)-12(3)-18(2)-22(1)'

In [92]:
tree.insert(4)
tree.print_dfs()

'4(1)-5(2)-8(3)-11(1)-12(4)-18(2)-22(1)'

In [93]:
tree.insert(7)
tree.print_dfs()

'4(1)-5(2)-7(1)-8(3)-11(1)-12(4)-18(2)-22(1)'

In [94]:
tree.insert(20)
tree.print_dfs()

RL


'4(1)-5(2)-7(1)-8(3)-11(1)-12(4)-18(1)-20(2)-22(1)'